## Python Advanced Code Stuff from James Powell talks
https://www.youtube.com/watch?v=L3j2qw9XZCchttps://www.youtube.com/watch?v=L3j2qw9XZCc

In [7]:
from sys import meta_path
from importlib import import_module
from sys import modules

### Path Hooks

In [8]:
# The class will hook into modules import and check if the import spelling is correct or not and will correct according to the dictionary values in spellcheck
class Hook:
    spellcheck = {'maht': 'math',
                    'randon': 'random'}
    def find_module(self, module, _):
        if module in self.spellcheck:
            return self
    def load_module(self, module):
        modules[module] = import_module(self.spellcheck[module])
        return modules[module]
meta_path.clear()
meta_path.append(Hook())

In [9]:
import maht
maht.sin

<function math.sin(x, /)>

In [10]:
import randon
randon.random

<function Random.random>

### Custom Hook named rename Module

In [13]:
from importlib import import_module
from sys import modules
from warnings import warn
from datetime import date, timedelta
from collections import namedtuple

class RenameHook:
    def __init__(self, registry):
        self.registry = registry
    
    def find_module(self, fullname, path = None):
        if fullname in self.registry:
            entry = self.registry[fullname]
            if len(entry) == 4:
                warning_date, exception_date, new_name, old_name = entry
            else:
                warning_date = date.today()
                exception_date = date.today() + timedelta(1)
                new_name = entry
            msg = ' '.join(['{} has been moved to {} as of {:%Y-%m-%d};',
                            'it will no longer be available after {:%Y-%m-%d};',
                            'Please update your imports'])
            msg = msg.format(fullname, new_name, warning_date, exception_date)
            if date.today() >= exception_date:
                raise ImportError(msg)
            elif date.today() >= warning_date:
                warn(msg, DeprecationWarning)
            return self

    def load_module(self, fullname):
        entry = self.registry[fullname]
        if len(entry) == 4:
            new_name = entry.new_name
        else:
            new_name = entry
        mod = import_module(new_name)
        modules[fullname] = mod
        return mod

RenamedModule = namedtuple('RenamedModule', 'from_date to_date new_name old_name')
    
if __name__ != '__main__':
    registry = {'moth': RenamedModule(date(2019, 7, 1), date(2020, 7, 1), 'math', 'moth'),
                'random': 'numpy.random',
                'linalg': 'numpy.linalg'}

    import sys
    sys.meta_path = [RenameHook(registry)] + sys.meta_path
         

In [ ]:
print("ehllow")